<a href="https://colab.research.google.com/github/Rabbia-Ijaz/SentiMix-Hindi-English/blob/main/Hindi_English_Sentimix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re 
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Data Cleaning Functions**

In [ ]:
#Data Cleaning functions

def RemoveNumbers(L): 
    pattern = '[0-9]'
    L = [re.sub(pattern, '', i) for i in L] 
    return L
def RemoveEngStopWords(Sentences):
  stop_words = set(stopwords.words('english'))  
  for i in range(0,len(Sentences)):
    word_tokens = word_tokenize(Sentences[i])  
    temp=""
    for w in word_tokens:  
        if w not in stop_words:  
            temp=temp+ w +" "
    Sentences[i]=temp
  return Sentences

def RemoveEmoji(Sentences):

  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
  for i in range(0,len(Sentences)):
    Sentences[i]=emoji_pattern.sub(r'', Sentences[i])
  return Sentences
def RemoveChar(Sentences):
  for i in range(0,len(Sentences)):
    Sentences[i] = re.sub(r'(https? \/\/)(\s)*([\w\-]+) (\s)*(/)*(\s)*([\w\-]+) (/s)*', '', Sentences[i], flags=re.MULTILINE)
    Sentences[i]=re.sub('[^A-Za-z0-9 /s]','',Sentences[i])
    Sentences[i]=re.sub('[0123456789\/®³“ë”ïž|£»–¶ï}{%^ã‹à°&*()ðŸ˜><‰™¤—.Šœ‡ªÿ‘"â€?_¥…!¦@#$,-]', '', Sentences[i])
    Sentences[i]=' '.join(Sentences[i].split())
  return Sentences


# **File Reading**

In [ ]:
#Reading Training Data
tweets = []
cur = {}
with open("/content/drive/MyDrive/Colab Notebooks/train_conll.txt", "r", encoding="utf-8") as f:
    for i in f:
        i = i.strip().lower()
        arr = i.split('\t')
        if len(arr)==3:
            tweets.append(cur)
            cur = {}
            cur['id'] = arr[1]
            if arr[2] == 'negative':
                cur['bias'] = -1
            elif arr[2] == 'positive':
                cur['bias'] = 1
            elif arr[2] == 'neutral':
                cur['bias'] = 0
            else:
                print('Error here')
                print(arr)
                break
        elif len(arr)==2:
            if 'tokens' in cur:
                cur['tokens'].append(arr)
            else:
                cur['tokens'] = [arr]

Sentences=[]
ID=[]
Labels=[]
column_names = ["Id", "Sentence", "Label"]
TDF = pd.DataFrame(columns = column_names)
for i in range (1,len(tweets)):
  temp=""
  ID.append(tweets[i]['id'])
  Labels.append(tweets[i]['bias'])
  k=len(tweets[i]['tokens'])
  for j in range (0,k):
    if (tweets[i]['tokens'])[j][1]!='o' and (tweets[i]['tokens'])[j][1]!='ent':
      temp=temp + ((tweets[i]['tokens'])[j][0]+' ')
  Sentences.append(temp)
  TDF.loc[i-1,'Sentence']=Sentences[i-1]
  TDF.loc[i-1,'Id']=ID[i-1]
  TDF.loc[i-1,'Label']=Labels[i-1]

#Data Cleaning for Train Data
Sentences = RemoveEngStopWords(Sentences)
Sentences = RemoveEmoji(Sentences)
Sentences = RemoveChar(Sentences)
Sentences = RemoveNumbers(Sentences)

#***************************************************************************#

#Reading Test Data
Testtweets = []
Testcur = {}
last="@	BhadasManKi @ mithleshkumarmi @ Kiranja34522516	@ BJPindia @ sherryontopp Chutiya h madarchod Sabse harami h ye thuk k â€¦ https // t	co /	0CcNA0NJqf"
with open("/content/drive/MyDrive/Colab Notebooks/Hindi_test_unalbelled_conll_updated.txt", "r", encoding="utf-8") as f:
    for i in f:
        i = i.strip().lower()
        arr = i.split('\t')
        #print(arr)
        if arr[0]=='meta':
          Testtweets.append(Testcur)
          Testcur = {}
          Testcur['id'] = arr[1]
        elif arr[0]!='meta':
          if 'tokens' in Testcur:
              Testcur['tokens'].append(arr)
          else:
              Testcur['tokens'] = [arr]
    

TestSentences=[]
TestID=[]
column_names = ["Id", "Sentence"]
TestTDF = pd.DataFrame(columns = column_names)
for i in range (1,len(Testtweets)):
  temp=""
  TestID.append(Testtweets[i]['id'])
  k=len(Testtweets[i]['tokens'])
  for j in range (0,k):      
    temp=temp + ((Testtweets[i]['tokens'])[j][0]+' ')
  TestSentences.append(temp)
  TestTDF.loc[i-1,'Sentence']=TestSentences[i-1]
  TestTDF.loc[i-1,'Id']=TestID[i-1]
TestSentences.append(last)

#Data Cleaning for Test Data
TestSentences = RemoveEngStopWords(TestSentences)
TestSentences = RemoveEmoji(TestSentences)
TestSentences = RemoveChar(TestSentences)
TestSentences = RemoveNumbers(TestSentences)


#***************************************************************************#

#Reading Test Labels
TestLabels = []
TestID=[]
with open("/content/drive/MyDrive/Colab Notebooks/test_labels_hinglish.txt", "r", encoding="utf-8") as f:
    for i in f:
        i = i.strip().lower()
        arr = i.split(',')
        if arr[0] !='uid':
          TestID.append(arr[0])
        if arr[1] == 'negative':
          TestLabels.append(-1)
        elif arr[1] == 'positive':
          TestLabels.append(1)
        elif arr[1] == 'neutral':
          TestLabels.append(0)

Y_Train,Y_Test=Labels,TestLabels

# **Models**

In [ ]:
Max=0
F_Pred=0
vectorizer = CountVectorizer(max_features=10000, min_df=6, max_df=0.1)
X_Train = vectorizer.fit_transform(Sentences).toarray()
X_Test=vectorizer.transform(TestSentences).toarray()

#Fitting the model
SGDC = SGDClassifier()
LSVC = LinearSVC()
MNB = MultinomialNB()
classifier = RandomForestClassifier(n_estimators=600, random_state=1)

classifier.fit(X_Train, Y_Train)
y_pred=classifier.predict(X_Test)
accuracy=metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy:
  Max=accuracy
  F_Pred=y_pred
print ('accuracy_score_randforest = '+str('{:04.2f}'.format(accuracy*100))+'%')

LSVC.fit(X_Train, Y_Train)
y_pred=LSVC.predict(X_Test)
accuracy_score_lsvc_CV = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_lsvc_CV:
  Max=accuracy_score_lsvc_CV
  F_Pred=y_pred
print('accuracy_score_lsvc_cv = '+str('{:4.2f}'.format(accuracy_score_lsvc_CV*100))+'%')

SGDC.fit(X_Train, Y_Train)
y_pred=SGDC.predict(X_Test)
accuracy_score_sgdc_CV = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_sgdc_CV:
  Max=accuracy_score_sgdc_CV
  F_Pred=y_pred
print('accuracy_score_sgdc_cv = '+str('{:4.2f}'.format(accuracy_score_sgdc_CV*100))+'%')

MNB.fit(X_Train, Y_Train)
y_pred=MNB.predict(X_Test)
accuracy_score_mnb = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_mnb:
  Max=accuracy_score_mnb
  F_Pred=y_pred
print('accuracy_score_mnb_cv = '+str('{:4.2f}'.format(accuracy_score_mnb*100))+'%')


accuracy_score_randforest = 66.27%


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy_score_lsvc_cv = 59.53%
accuracy_score_sgdc_cv = 61.90%
accuracy_score_mnb_cv = 65.33%


In [ ]:
vectorizer = TfidfVectorizer()
X_Train = vectorizer.fit_transform(Sentences).toarray()
X_Test=vectorizer.transform(TestSentences).toarray()

#on TF-IDF data
LSVC.fit(X_Train, Y_Train)
y_pred=LSVC.predict(X_Test)
accuracy_score_lsvc = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_lsvc:
  Max=accuracy_score_lsvc
  F_Pred=y_pred
print('accuracy_score_lsvc = '+str('{:4.2f}'.format(accuracy_score_lsvc*100))+'%')

SGDC.fit(X_Train, Y_Train)
y_pred=SGDC.predict(X_Test)
accuracy_score_sgdc = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_sgdc:
  Max=accuracy_score_sgdc
print('accuracy_score_sgdc = '+str('{:4.2f}'.format(accuracy_score_sgdc*100))+'%')

MNB.fit(X_Train, Y_Train)
y_pred=MNB.predict(X_Test)
accuracy_score_mnb = metrics.accuracy_score(y_pred, Y_Test)
if Max<accuracy_score_mnb:
  Max=accuracy_score_mnb
  F_Pred=y_pred
print('accuracy_score_mnb = '+str('{:4.2f}'.format(accuracy_score_mnb*100))+'%')

print (Max)

accuracy_score_lsvc = 63.83%
accuracy_score_sgdc = 65.53%
accuracy_score_mnb = 65.50%
0.6626666666666666


In [ ]:
File='/content/drive/MyDrive/Colab Notebooks/answer.txt'
f = open(File, "w")
f.write("Uid,Sentiment\n")
#"This is line %d\r" % (i+1)
i=0
for i in range(0,len(TestID)-1):
  if F_Pred[i]== 1:
    f.write(str(TestID[i])+",positive\n")
  elif F_Pred[i]==-1:
    f.write(str(TestID[i])+",negative\n")
  elif F_Pred[i]==0:
    f.write(str(TestID[i])+",neutral\n")
i=len(TestID)-1
if F_Pred[i]== 1:
  f.write(str(TestID[i])+",positive")
elif F_Pred[i]==-1:
  f.write(str(TestID[i])+",negative")
elif F_Pred[i]==0:
  f.write(str(TestID[i])+",neutral")
f.close()

#open and read the file after the appending:
#f = open("/content/drive/MyDrive/Colab Notebooks/answer.txt", "r")
#print(f.read())
